In [ ]:
#YOU CAN DESCONSIDER THIS... TRYING TO MAKE A SCRIPT TO GET THE DATA FROM THE API MANUALLY, BUT NOW THE THING IS WORKING AGAIN
#SO JUST USING THE OTHER NOTEBOOK

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from polygon import RESTClient
import datetime as dt
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os
import requests
import aiohttp
import asyncio
from datetime import datetime, timedelta

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")


In [3]:
tickers=['AAPL']

In [7]:
data_fetcher.get_urls('AAPL')

['https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-01-01/2003-09-18?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-09-19/2004-06-05?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2004-06-06/2005-02-21?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2005-02-22/2005-11-09?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2005-11-10/2006-07-28?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2006-07-29/2007-04-15?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/r

In [4]:
class Fetching_Data:
    def __init__(self, tickers, trading_hours=16, start_date='2003-01-01', end_date='2024-02-01', timespan='minute', multiplier=5, limit=50000, api_key=api_key):
        self.tickers = tickers
        self.trading_hours = trading_hours
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
        self.timespan = timespan
        self.multiplier = multiplier
        self.limit = limit
        self.api_key = api_key

    def calculate_range(self): # THIS IS WORKING FINE
        if self.timespan == 'minute':
            return (self.limit // (self.trading_hours * 60)) * self.multiplier
        elif self.timespan == 'hour':
            return (self.limit // self.trading_hours) * self.multiplier
        elif self.timespan == 'day':
            return self.limit * self.multiplier

    def get_urls(self, ticker): # this is also working fine
        urls = []
        days_to_add = self.calculate_range()
        from_date = self.start_date
        while from_date < self.end_date:
            to_date = from_date + timedelta(days=days_to_add)
            if to_date > self.end_date:
                to_date = self.end_date
            url = f'https://api.polygon.io/v2/aggs/ticker/{ticker.strip()}/range/{self.multiplier}/{self.timespan}/{from_date.strftime("%Y-%m-%d")}/{to_date.strftime("%Y-%m-%d")}?adjusted=true&sort=asc&limit={self.limit}&apiKey={self.api_key}'
            urls.append(url)
            from_date = to_date + timedelta(days=1)
        return urls

    def fetch_data(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            return None


    def get_data(self):
        for ticker in self.tickers:
            urls = self.get_urls(ticker)
            all_data = pd.DataFrame()  # Initialize an empty DataFrame to store all data.
            for url in urls:
                data = self.fetch_data(url)
                if data and 'results' in data:  # Ensure there are results
                    df = pd.DataFrame(data['results'])
                    df['t'] = pd.to_datetime(df['t'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
                    df = df.rename(columns={'t': 'Date'}).set_index('Date')
                    df.drop(columns=['vw', 'o', 'h', 'l', 'v', 'n'], inplace=True)  # Adjust columns as needed
                    all_data = pd.concat([all_data, df])  # Append each DataFrame to the all_data DataFrame

            all_data.to_csv(f"{ticker.strip()}_data.csv", index=True)  # Save the combined DataFrame to CSV



                    


In [ ]:
# class Fetching_Data:
#     def __init__(self, tickers, trading_hours=16, start_date='2003-01-01', end_date='2024-02-01', timespan='minute', multiplier=5, limit=50000, api_key=api_key):
#         self.tickers = tickers
#         self.trading_hours = trading_hours
#         self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
#         self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
#         self.timespan = timespan
#         self.multiplier = multiplier
#         self.limit = limit
#         self.api_key = api_key

#     def calculate_range(self): # THIS IS WORKING FINE
#         if self.timespan == 'minute':
#             return (self.limit // (self.trading_hours * 60)) * self.multiplier
#         elif self.timespan == 'hour':
#             return (self.limit // self.trading_hours) * self.multiplier
#         elif self.timespan == 'day':
#             return self.limit * self.multiplier

#     def get_urls(self, ticker): # this is also working fine
#         urls = []
#         days_to_add = self.calculate_range()
#         from_date = self.start_date
#         while from_date < self.end_date:
#             to_date = from_date + timedelta(days=days_to_add)
#             if to_date > self.end_date:
#                 to_date = self.end_date
#             url = f'https://api.polygon.io/v2/aggs/ticker/{ticker.strip()}/range/{self.multiplier}/{self.timespan}/{from_date.strftime("%Y-%m-%d")}/{to_date.strftime("%Y-%m-%d")}?adjusted=true&sort=asc&limit={self.limit}&apiKey={self.api_key}'
#             urls.append(url)
#             from_date = to_date + timedelta(days=1)
#         return urls

#     async def fetch_data(self, session, url):
#         async with session.get(url) as response:
#             data = await response.json()
#             return data



#     async def get_data_async(self):
#         async with aiohttp.ClientSession() as session:
#             tasks = []
#             for ticker in self.tickers:
#                 urls = self.get_urls(ticker)
#                 for url in urls:
#                     task = asyncio.create_task(self.fetch_data(session, url))
#                     tasks.append(task)
#             responses = await asyncio.gather(*tasks)
#             # Process data into dataframes and save to CSV
#             for ticker, response in zip(self.tickers * len(urls), responses):
#                 if 'results' in response:  # Ensure there are results
#                     df = pd.DataFrame(response['results'])
#                     df['t'] = pd.to_datetime(df['t'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
#                     df = df.rename(columns={'t': 'Date'}).set_index('Date')
#                     df.drop(columns=['vw', 'o', 'h', 'l', 'v', 'n'], inplace=True)
#                     df.to_csv(f"{ticker.strip()}_data.csv", index=False)
                    
                    
                    




In [5]:
class DataPipeline:
    def __init__(self, tickers, start_date='2003-01-01', end_date='2024-02-01', timespan='minute', multiplier=5, limit=50000):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.timespan = timespan
        self.multiplier = multiplier
        self.limit = limit
    
    def load_csv_data(self, ticker):
        
        try:
            data = pd.read_csv(f"{ticker.strip()}_data.csv")
            data.index = pd.to_datetime(data['t'], unit='ms').tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
            data = data.rename(columns={'timestamp': 'Date'})
            filtered_data = data.between_time('09:30', '16:00').copy()
            # data = data.drop(columns=['o', 'vw', 'h', 'l', 'v', 't', 'n'])
            return data, filtered_data
        except FileNotFoundError:
            print(f"No CSV file found for {ticker}")
            return pd.DataFrame(), pd.DataFrame()

    def process_data(self, df):
        df['Day'] = pd.to_datetime(df.index).normalize()
        df['returns'] = df['close'].pct_change()
        df['log_ret'] = np.log(df['close']).diff()
        df['squared_ret'] = df['returns']**2
        df['squared_log_ret'] = df['log_ret']**2
        return df
    
    def calculate_realized_var(self, df):
        realized_var = df.groupby('Day').agg({'squared_ret': 'sum',
                                              'squared_log_ret': 'sum'})
        return realized_var.rename(columns={'squared_ret': 'daily_realised_var',
                                            'squared_log_ret': 'daily_realised_log_var'})

    def processing_pipeline(self):
        results = {}
        for ticker in self.tickers:
            data, filtered_data = self.load_csv_data(ticker)
            processed_data = self.process_data(data)
            processed_filtered_data = self.process_data(filtered_data)
            processed_data.to_csv(f'{ticker}_processed_data.csv')
            processed_filtered_data.to_csv(f'{ticker}_filtered_data_processed.csv')
            realized_var = self.calculate_realized_var(processed_data)
            realized_var_filtered = self.calculate_realized_var(processed_filtered_data).add_prefix('filtered_')
            final_var = realized_var.join(realized_var_filtered, how='outer')
            final_var.to_csv(f'{ticker}_realized_var.csv')
            results[ticker] = [realized_var, realized_var_filtered]
        return results


In [6]:
tickers=['AAPL']
data_fetcher = Fetching_Data(tickers)

# result = data_fetcher.get_urls()


In [8]:
b = pd.read_csv('/Users/raphaelravinet/Code/BSE/Thesis/data/AAPL.csv')

In [13]:
b.rename(columns={'timestamp': 'Date'}, inplace=True)

In [9]:
a = pd.read_csv('AAPL_data.csv')

In [ ]:
pipeline = DataPipeline(tickers)
results = pipeline.processing_pipeline()

In [14]:
from datetime import datetime

# Assuming 'b' is your DataFrame with the 'Date' column containing timestamps with timezone
b['Date'] = b['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z').strftime('%Y-%m-%d %H:%M:%S'))

In [16]:
data_fetcher.get_urls('AAPL')[0]

['https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-01-01/2003-09-18?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-09-19/2004-06-05?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2004-06-06/2005-02-21?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2005-02-22/2005-11-09?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2005-11-10/2006-07-28?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2006-07-29/2007-04-15?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/r

In [21]:
try_url = data_fetcher.get_urls('AAPL')[0]

In [27]:
try_url

'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-01-01/2003-09-18?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v'

In [23]:
response_try = requests.get(try_url)


In [26]:
pd.DataFrame(response_try.json())

,ticker,queryCount,resultsCount,adjusted,results,status,request_id,count
0,AAPL,2848,628,True,"{'v': 5600, 'vw': 0.3994, 'o': 0.3994, 'c': 0....",OK,e8898f506d08b3b09d4374c21f6268a6,628
1,AAPL,2848,628,True,"{'v': 22400, 'vw': 0.3982, 'o': 0.3982, 'c': 0...",OK,e8898f506d08b3b09d4374c21f6268a6,628
2,AAPL,2848,628,True,"{'v': 67200, 'vw': 0.3977, 'o': 0.3977, 'c': 0...",OK,e8898f506d08b3b09d4374c21f6268a6,628
3,AAPL,2848,628,True,"{'v': 218400, 'vw': 0.3971, 'o': 0.3973, 'c': ...",OK,e8898f506d08b3b09d4374c21f6268a6,628
4,AAPL,2848,628,True,"{'v': 5784352.0, 'vw': 0.398, 'o': 0.3973, 'c'...",OK,e8898f506d08b3b09d4374c21f6268a6,628
...,...,...,...,...,...,...,...,...
623,AAPL,2848,628,True,"{'v': 16800, 'vw': 0.4084, 'o': 0.4084, 'c': 0...",OK,e8898f506d08b3b09d4374c21f6268a6,628
624,AAPL,2848,628,True,"{'v': 207200, 'vw': 0.4086, 'o': 0.4086, 'c': ...",OK,e8898f506d08b3b09d4374c21f6268a6,628
625,AAPL,2848,628,True,"{'v': 22400, 'vw': 0.4086, 'o': 0.4086, 'c': 0...",OK,e8898f506d08b3b09d4374c21f6268a6,628
626,AAPL,2848,628,True,"{'v': 16800, 'vw': 0.4086, 'o': 0.4086, 'c': 0...",OK,e8898f506d08b3b09d4374c21f6268a6,628


In [17]:
# Assuming both 'a' and 'b' DataFrames have a 'Date' column containing timestamps
# Convert 'Date' column in DataFrame 'a' to string for comparison
a['Date'] = a['Date'].astype(str)

# Check which indices in DataFrame 'b' are not present in DataFrame 'a'
indices_not_in_a = b[~b['Date'].isin(a['Date'])].index

# Now 'indices_not_in_a' contains the indices in DataFrame 'b' that are not in DataFrame 'a'
# Get the actual dates corresponding to these indices
dates_not_in_a = b.loc[indices_not_in_a, 'Date']

# Now 'dates_not_in_a' contains the dates from DataFrame 'b' that are not in DataFrame 'a'
print(dates_not_in_a)


11397     2004-03-12 13:45:00
11398     2004-03-12 13:50:00
11399     2004-03-12 13:55:00
11400     2004-03-12 14:00:00
11401     2004-03-12 14:05:00
                 ...         
787273    2023-09-21 19:35:00
787274    2023-09-21 19:40:00
787275    2023-09-21 19:45:00
787276    2023-09-21 19:50:00
787277    2023-09-21 19:55:00
Name: Date, Length: 445236, dtype: object


In [19]:
pd.DataFrame(dates_not_in_a)

,Date
11397,2004-03-12 13:45:00
11398,2004-03-12 13:50:00
11399,2004-03-12 13:55:00
11400,2004-03-12 14:00:00
11401,2004-03-12 14:05:00
...,...
787273,2023-09-21 19:35:00
787274,2023-09-21 19:40:00
787275,2023-09-21 19:45:00
787276,2023-09-21 19:50:00


In [ ]:
a

In [ ]:
# class DataPipeline:
#     def __init__(self, client, tickers, start_date = '1970-01-01', end_date = '2024-02-01', timespan = 'minute', multiplier = 5, limit = 50000):
#         self.client = client
#         self.tickers = tickers
#         self.start_date = start_date
#         self.end_date = end_date
#         self.timespan = timespan
#         self.multiplier = multiplier
#         self.limit = limit
    
#    def load_csv_data(self, ticker):
#         try:
#             data = pd.read_csv(f"{ticker.strip()}_data.csv", index_col='timestamp', parse_dates=True)
#             data.index = pd.to_datetime(data.index, unit='ms').tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
#             data = data.rename(columns={'timestamp': 'Date'}).set_index('Date')
#             filtered_data = data.between_time('09:30', '16:00')
#             filtered_data = filtered_data.drop(columns=['open', 'vwap', 'high', 'low', 'volume', 'transactions', 'otc'])
#             data = data.drop(columns=['open', 'vwap', 'high', 'low', 'volume', 'transactions', 'otc'])
#             return data, filtered_data
#         except FileNotFoundError:
#             print(f"No CSV file found for {ticker}")
#             return pd.DataFrame(), pd.DataFrame()

   
            
            
#     def process_data(self,df):
#         df['Day'] = pd.to_datetime(df.index).normalize()
#         df['returns'] = df['close'].pct_change()
#         df['log_ret'] = np.log(df['close']).diff()
#         df['squared_ret'] = df['returns']**2
#         df['squared_log_ret'] = df['log_ret']**2
#         return df
    
#     def calculate_realized_var(self,df):
#         realized_var = df.groupby('Day').agg({'squared_ret': 'sum',
#                                               'squared_log_ret': 'sum'})
#         return realized_var.rename(columns = {'squared_ret': 'daily_realised_var',
#                                         'squared_log_ret': 'daily_realised_log_var'})

    
#     def processing_pipeline(self):
#         results = {}
#         for ticker in self.tickers:
#             data, filtered_data = self.load_csv_data(ticker)
#             processed_data = self.process_data(data)
#             processed_filtered_data = self.process_data(filtered_data)
#             processed_data.to_csv(f'{ticker}_data.csv')
#             processed_filtered_data.to_csv(f'{ticker}_filtered_data.csv')
#             realized_var = self.calculate_realized_var(processed_data)
#             realized_var_filtered = self.calculate_realized_var(processed_filtered_data).add_prefix('filtered_')
#             realized_var.join(realized_var_filtered, how = 'outer').to_csv(f'{ticker}_realized_var.csv')
#             results[ticker] = [realized_var, realized_var_filtered]
#             return results
        
